In [53]:
import pandas as pd
import numpy as np

In [54]:
df1 = pd.read_parquet('presentation_data/yellow_tripdata_2023-01.parquet', engine='pyarrow')
df2 = pd.read_parquet('presentation_data/yellow_tripdata_2023-02.parquet', engine='pyarrow')
df3 = pd.read_parquet('presentation_data/yellow_tripdata_2023-03.parquet', engine='pyarrow')
df4 = pd.read_parquet('presentation_data/yellow_tripdata_2023-04.parquet', engine='pyarrow')
df5 = pd.read_parquet('presentation_data/yellow_tripdata_2023-05.parquet', engine='pyarrow')

In [ ]:
df_all = pd.concat([df1, df2, df3, df4, df5], axis=0)

In [ ]:
print(max(df_all['PULocationID']))

265


In [ ]:
df_all.shape

(16186386, 20)

In [ ]:
df_all.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'Airport_fee'],
      dtype='object')

In [ ]:
cross_tab = pd.crosstab(df_all['PULocationID'], df_all['DOLocationID'], normalize='all')

# Convert column names to integers
columns = sorted(map(int, cross_tab.columns))

# Find missing columns and insert them with zeros
for prev, curr in zip(columns, columns[1:]):
    if curr - prev > 1:
        missing_columns = [str(col) for col in range(prev + 1, curr)]
        for col in missing_columns:
            cross_tab[col] = 0

# Sort columns to ensure they are in order
cross_tab = cross_tab[sorted(cross_tab.columns, key=int)]




index_values = sorted(map(int, cross_tab.index))

# Find missing rows and insert them with zeros
for prev, curr in zip(index_values, index_values[1:]):
    if curr - prev > 1:
        missing_rows = [index for index in range(prev + 1, curr)]
        for index in missing_rows:
            cross_tab.loc[index] = 0

# Sort the DataFrame based on index
cross_tab = cross_tab.sort_index()

cross_tab.shape



(265, 265)

In [ ]:
matrix_normalized = cross_tab.to_numpy()

In [ ]:
output_file_npy = 'Data/normalized_adj_matrix.npy'
np.save(output_file_npy, matrix_normalized)

## Data Matrix

In [ ]:
columns_to_keep = ['tpep_pickup_datetime', 'PULocationID']
df_all_simpl = df_all[columns_to_keep]

In [ ]:
#df_all.to_csv('Data/df_all.csv')

In [ ]:
df_all_simpl['tpep_pickup_datetime'] = pd.to_datetime(df_all_simpl['tpep_pickup_datetime'])

df_all_simpl['interval'] = df_all_simpl['tpep_pickup_datetime'].dt.floor('30T')

grouped_data = df_all_simpl.groupby(['PULocationID', 'interval']).size().reset_index(name='count')


taxi_matrix = grouped_data.pivot(index='interval', columns='PULocationID', values='count').fillna(0)


taxi_matrix.index = taxi_matrix.index.strftime('%Y-%m-%d %H:%M:%S')


print(taxi_matrix.shape)

# Convert column names to integers
columns = sorted(map(int, taxi_matrix.columns))

# Find missing columns and insert them with zeros
for prev, curr in zip(columns, columns[1:]):
    if curr - prev > 1:
        missing_columns = [str(col) for col in range(prev + 1, curr)]
        for col in missing_columns:
            taxi_matrix[col] = 0

# Sort columns to ensure they are in order
taxi_matrix = taxi_matrix[sorted(taxi_matrix.columns, key=int)]
print(taxi_matrix.shape)


/var/folders/nv/yh00rxr94090cfx1pg6840p40000gn/T/ipykernel_88838/4139302126.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_simpl['tpep_pickup_datetime'] = pd.to_datetime(df_all_simpl['tpep_pickup_datetime'])
/var/folders/nv/yh00rxr94090cfx1pg6840p40000gn/T/ipykernel_88838/4139302126.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_simpl['interval'] = df_all_simpl['tpep_pickup_datetime'].dt.floor('30T')


(7306, 262)
(7306, 265)


In [ ]:
taxi_matrix = taxi_matrix.to_numpy()
output_file_npy = 'Data/data_matrix_bench.npy'
np.save(output_file_npy, taxi_matrix)

## LSTM

In [ ]:
check = lambda x: 1 if x.weekday() == 4 and x.hour >= 23 or x.weekday() == 5 and x.hour < 5 or x.weekday() == 5 and x.hour >= 23 or x.weekday() == 6 and x.hour < 5 else 0

def df_to_X_y2(df, window_size):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np) - window_size):
        row = [r for r in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size][0]
        y.append(label)
    return np.array(X), np.array(y)

def preproc(X):
    X[:,:,0] = (X[:,:,0] - training_mean) / training_std
    return X

def mape(actual, predicted):
    mask = actual != 0
    return (np.fabs(actual - predicted)/actual)[mask].mean() * 100





2023-12-18 01:03:11.474897: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-12-18 01:03:11.477368: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-12-18 01:03:11.478752: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 7306, 265)]  0           []                               
                                                                                                  
 lstm_2 (LSTM)                  (None, 7306, 64)     84480       ['input_5[0][0]']                
                                                                                                  
 input_6 (InputLayer)           [(None, 265, 265)]   0           []                               
                                                                                                  
 flatten_4 (Flatten)            (None, 467584)       0           ['lstm_2[0][0]']                 
                                                                                            

ValueError: Training data contains 1 samples, which is not sufficient to split it into a validation and training set as specified by `validation_split=0.2`. Either provide more data, or a different value for the `validation_split` argument.